<a href="https://colab.research.google.com/github/hieuphanuit/Coursera_Capstone/blob/master/final.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# IBM Applied Data Science Capstone Course by Coursera
### Week 5 Final Report
**_Opening a New Shopping Mall in HCM city_**
***
### 1. Import libraries

In [ ]:
!pip install geocoder

In [3]:
import numpy as np # library to handle data in a vectorized manner

import pandas as pd # library for data analsysis
pd.set_option("display.max_columns", None)
pd.set_option("display.max_rows", None)

import json # library to handle JSON files

from geopy.geocoders import Nominatim # convert an address into latitude and longitude values
import geocoder # to get coordinates

import requests # library to handle requests
from bs4 import BeautifulSoup # library to parse HTML and XML documents

from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

import folium # map rendering library

print("Libraries imported.")

Libraries imported.


### 2. Scrap data from Wikipedia page into a DataFrame

In [4]:
# send the GET request
data = requests.get("https://en.wikipedia.org/wiki/Category:Districts_of_Ho_Chi_Minh_City").text

In [5]:
# parse data from the html into a beautifulsoup object
soup = BeautifulSoup(data, 'html.parser')

In [6]:
# create a list to store neighborhood data
neighborhoodList = []

In [7]:
# append the data into the list
for row in soup.find_all("div", class_="mw-category")[0].findAll("li"):
    neighborhoodList.append(row.text)

In [9]:
# create a new DataFrame from the list
df = pd.DataFrame({"Neighborhood": neighborhoodList})

df.head()

,Neighborhood
0,Template:List of HCMC Administrative Units
1,Bình Chánh District
2,"Bình Tân District, Ho Chi Minh City"
3,Bình Thạnh District
4,Cần Giờ District


In [10]:
# print the number of rows of the dataframe
df.shape

(23, 1)

### 3. Get the geographical coordinates

In [11]:
# define a function to get coordinates
def get_latlng(neighborhood):
    # initialize your variable to None
    lat_lng_coords = None
    # loop until you get the coordinates
    while(lat_lng_coords is None):
        g = geocoder.arcgis('{}, Ho Chi Minh, Viet Nam'.format(neighborhood))
        lat_lng_coords = g.latlng
    return lat_lng_coords

In [12]:
# call the function to get the coordinates, store in a new list using list comprehension
coords = [ get_latlng(neighborhood) for neighborhood in kl_df["Neighborhood"].tolist() ]

In [13]:
coords

[[10.776410000000055, 106.70111000000009],
 [10.679220000000043, 106.57654000000008],
 [10.73684000000003, 106.61448000000007],
 [10.806080000000065, 106.69297000000006],
 [10.41566000000006, 106.96130000000005],
 [10.977340000000027, 106.50223000000005],
 [10.78096000000005, 106.69911000000008],
 [10.775650000000041, 106.68672000000004],
 [10.766700000000071, 106.70647000000008],
 [10.755690000000072, 106.66637000000009],
 [10.745970000000057, 106.64769000000007],
 [10.70515000000006, 106.73748000000006],
 [10.74771000000004, 106.66334000000006],
 [10.768830000000037, 106.66599000000008],
 [10.763160000000028, 106.64314000000007],
 [10.850440000000049, 106.62731000000008],
 [10.833790000000022, 106.66556000000008],
 [10.888360000000034, 106.59640000000007],
 [10.701530000000048, 106.73818000000006],
 [10.795650000000023, 106.67464000000007],
 [10.73684000000003, 106.61448000000007],
 [10.782320000000027, 106.63667000000004],
 [10.861779986287589, 106.79610692772711]]

In [14]:
# create temporary dataframe to populate the coordinates into Latitude and Longitude
df_coords = pd.DataFrame(coords, columns=['Latitude', 'Longitude'])

In [15]:
# merge the coordinates into the original dataframe
df['Latitude'] = df_coords['Latitude']
df['Longitude'] = df_coords['Longitude']

In [17]:
# check the neighborhoods and the coordinates
print(df.shape)
df

(23, 3)


,Neighborhood,Latitude,Longitude
0,Template:List of HCMC Administrative Units,10.77641,106.701110
1,Bình Chánh District,10.67922,106.576540
2,"Bình Tân District, Ho Chi Minh City",10.73684,106.614480
3,Bình Thạnh District,10.80608,106.692970
4,Cần Giờ District,10.41566,106.961300
5,Củ Chi District,10.97734,106.502230
6,"District 1, Ho Chi Minh City",10.78096,106.699110
7,"District 3, Ho Chi Minh City",10.77565,106.686720
8,"District 4, Ho Chi Minh City",10.76670,106.706470
9,"District 5, Ho Chi Minh City",10.75569,106.666370


In [18]:
# save the DataFrame as CSV file
df.to_csv("df.csv", index=False)

### 4. Create a map of Kuala Lumpur with neighborhoods superimposed on top

In [21]:
# get the coordinates of HCM
address = 'Ho Chi Minh, Viet Nam'

geolocator = Nominatim(user_agent="my-application")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Ho Chi Minh, Vietnam {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Ho Chi Minh, Vietnam 10.7758439, 106.7017555.


In [22]:
# create map of Toronto using latitude and longitude values
map_hcmc = folium.Map(location=[latitude, longitude], zoom_start=11)

# add markers to map
for lat, lng, neighborhood in zip(df['Latitude'], df['Longitude'], df['Neighborhood']):
    label = '{}'.format(neighborhood)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7).add_to(map_hcmc)  
    
map_hcmc

### 5. Use the Foursquare API to explore the neighborhoods

In [23]:
# define Foursquare Credentials and Version
CLIENT_ID = '' # your Foursquare ID
CLIENT_SECRET = '' # your Foursquare Secret
VERSION = '20210418' # Foursquare API version

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: HM1ULLVTXBQFWS4WK0EUI13QSEQY004URL5P5FPZ4SH5FWFV
CLIENT_SECRET:O0VILFVBFGTMQUD4MVOJDXT2ZFIM14FEM3FX3JADCAMIAHLU


**Now, let's get the top 100 venues that are within a radius of 2000 meters.**

In [25]:
radius = 2000
LIMIT = 100

venues = []

for lat, long, neighborhood in zip(df['Latitude'], df['Longitude'], df['Neighborhood']):
    
    # create the API request URL
    url = "https://api.foursquare.com/v2/venues/explore?client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}".format(
        CLIENT_ID,
        CLIENT_SECRET,
        VERSION,
        lat,
        long,
        radius, 
        LIMIT)
    
    # make the GET request
    results = requests.get(url).json()["response"]['groups'][0]['items']
    
    # return only relevant information for each nearby venue
    for venue in results:
        venues.append((
            neighborhood,
            lat, 
            long, 
            venue['venue']['name'], 
            venue['venue']['location']['lat'], 
            venue['venue']['location']['lng'],  
            venue['venue']['categories'][0]['name']))

In [26]:
# convert the venues list into a new DataFrame
venues_df = pd.DataFrame(venues)

# define the column names
venues_df.columns = ['Neighborhood', 'Latitude', 'Longitude', 'VenueName', 'VenueLatitude', 'VenueLongitude', 'VenueCategory']

print(venues_df.shape)
venues_df.head()

(1090, 7)


,Neighborhood,Latitude,Longitude,VenueName,VenueLatitude,VenueLongitude,VenueCategory
0,Template:List of HCMC Administrative Units,10.77641,106.70111,B3 - Steakhouse & Craft Beer,10.775190,106.702492,Steakhouse
1,Template:List of HCMC Administrative Units,10.77641,106.70111,Park Hyatt Saigon,10.777574,106.703609,Hotel
2,Template:List of HCMC Administrative Units,10.77641,106.70111,Pasteur Street Brewing Company,10.775220,106.700894,Brewery
3,Template:List of HCMC Administrative Units,10.77641,106.70111,Liberty Central Saigon Citypoint Hotel,10.774758,106.700795,Hotel
4,Template:List of HCMC Administrative Units,10.77641,106.70111,Cộng Cà Phê,10.777917,106.701266,Café


**Let's check how many venues were returned for each neighorhood**

In [27]:
venues_df.groupby(["Neighborhood"]).count()

,Latitude,Longitude,VenueName,VenueLatitude,VenueLongitude,VenueCategory
Neighborhood,,,,,,
Bình Chánh District,6,6,6,6,6,6
Bình Thạnh District,72,72,72,72,72,72
"Bình Tân District, Ho Chi Minh City",17,17,17,17,17,17
Cần Giờ District,4,4,4,4,4,4
Củ Chi District,5,5,5,5,5,5
"District 1, Ho Chi Minh City",100,100,100,100,100,100
"District 10, Ho Chi Minh City",100,100,100,100,100,100
"District 11, Ho Chi Minh City",56,56,56,56,56,56
"District 12, Ho Chi Minh City",8,8,8,8,8,8


**Let's find out how many unique categories can be curated from all the returned venues**

In [28]:
print('There are {} uniques categories.'.format(len(venues_df['VenueCategory'].unique())))

There are 125 uniques categories.


In [29]:
# print out the list of categories
venues_df['VenueCategory'].unique()[:50]

array(['Steakhouse', 'Hotel', 'Brewery', 'Café', 'Department Store',
       'Bar', 'Multiplex', 'Spanish Restaurant', 'Pizza Place',
       'Speakeasy', 'Supermarket', 'Vegetarian / Vegan Restaurant',
       'German Restaurant', 'French Restaurant', 'Clothing Store',
       'Italian Restaurant', 'Jazz Club', 'Massage Studio', 'Coffee Shop',
       'Thai Restaurant', 'Vietnamese Restaurant',
       'North Indian Restaurant', 'Japanese Restaurant', 'Cupcake Shop',
       'Bookstore', 'Cocktail Bar', 'Opera House', 'Jewelry Store',
       'Whisky Bar', 'Hawaiian Restaurant', 'Indian Restaurant',
       'Hotel Bar', 'Museum', 'Dessert Shop', 'Middle Eastern Restaurant',
       'Spa', 'Ramen Restaurant', 'Pub', 'Sushi Restaurant', 'Lounge',
       'Tapas Restaurant', 'Seafood Restaurant', 'Burger Joint',
       'Noodle House', 'Video Game Store', 'Sandwich Place', 'Fountain',
       'Convenience Store', 'Shoe Store', "Women's Store"], dtype=object)

In [ ]:
# check if the results contain "Shopping Mall"
"Neighborhood" in venues_df['VenueCategory'].unique()

True

### 6. Analyze Each Neighborhood

In [30]:
# one hot encoding
hcmc_onehot = pd.get_dummies(venues_df[['VenueCategory']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
hcmc_onehot['Neighborhoods'] = venues_df['Neighborhood'] 

# move neighborhood column to the first column
fixed_columns = [hcmc_onehot.columns[-1]] + list(hcmc_onehot.columns[:-1])
hcmc_onehot = hcmc_onehot[fixed_columns]

print(hcmc_onehot.shape)
hcmc_onehot.head()

(1090, 126)


,Neighborhoods,Airport Food Court,Airport Lounge,Airport Service,American Restaurant,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,BBQ Joint,Bakery,Bar,Basketball Court,Basketball Stadium,Beach,Bed & Breakfast,Beer Bar,Beer Garden,Bistro,Boat or Ferry,Bookstore,Boutique,Breakfast Spot,Brewery,Burger Joint,Bus Station,Café,Cantonese Restaurant,Chinese Restaurant,Clothing Store,Cocktail Bar,Coffee Shop,Convenience Store,Convention Center,Cupcake Shop,Department Store,Design Studio,Dessert Shop,Dim Sum Restaurant,Diner,Dumpling Restaurant,Electronics Store,Fabric Shop,Fast Food Restaurant,Flea Market,Flower Shop,Food,Food Court,Food Truck,Fountain,French Restaurant,Fried Chicken Joint,Gastropub,German Restaurant,Gift Shop,Golf Course,Grocery Store,Gym,Gym / Fitness Center,Hawaiian Restaurant,Hostel,Hotel,Hotel Bar,Hotpot Restaurant,Ice Cream Shop,Indian Restaurant,Italian Restaurant,Japanese Restaurant,Jazz Club,Jewelry Store,Juice Bar,Karaoke Bar,Korean Restaurant,Lounge,Market,Massage Studio,Mattress Store,Mexican Restaurant,Middle Eastern Restaurant,Movie Theater,Multiplex,Museum,Music Venue,Noodle House,North Indian Restaurant,Opera House,Outdoors & Recreation,Park,Pier,Pizza Place,Pub,Ramen Restaurant,Residential Building (Apartment / Condo),Rest Area,Restaurant,Russian Restaurant,Sandwich Place,Seafood Restaurant,Shoe Store,Shopping Mall,Shopping Plaza,Snack Place,Soup Place,Spa,Spanish Restaurant,Speakeasy,Steakhouse,Supermarket,Sushi Restaurant,Tapas Restaurant,Tattoo Parlor,Tea Room,Temple,Thai Restaurant,Theme Park,Track Stadium,Travel Agency,Turkish Restaurant,University,Vegetarian / Vegan Restaurant,Video Game Store,Vietnamese Restaurant,Warehouse Store,Water Park,Whisky Bar,Women's Store,Yoga Studio
0,Template:List of HCMC Administrative Units,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,Template:List of HCMC Administrative Units,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,Template:List of HCMC Administrative Units,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,Template:List of HCMC Administrative Units,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4,Template:List of HCMC Administrative Units,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


**Next, let's group rows by neighborhood and by taking the mean of the frequency of occurrence of each category**

In [31]:
hcmc_grouped = hcmc_onehot.groupby(["Neighborhoods"]).mean().reset_index()

print(hcmc_grouped.shape)
hcmc_grouped

(23, 126)


,Neighborhoods,Airport Food Court,Airport Lounge,Airport Service,American Restaurant,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,BBQ Joint,Bakery,Bar,Basketball Court,Basketball Stadium,Beach,Bed & Breakfast,Beer Bar,Beer Garden,Bistro,Boat or Ferry,Bookstore,Boutique,Breakfast Spot,Brewery,Burger Joint,Bus Station,Café,Cantonese Restaurant,Chinese Restaurant,Clothing Store,Cocktail Bar,Coffee Shop,Convenience Store,Convention Center,Cupcake Shop,Department Store,Design Studio,Dessert Shop,Dim Sum Restaurant,Diner,Dumpling Restaurant,Electronics Store,Fabric Shop,Fast Food Restaurant,Flea Market,Flower Shop,Food,Food Court,Food Truck,Fountain,French Restaurant,Fried Chicken Joint,Gastropub,German Restaurant,Gift Shop,Golf Course,Grocery Store,Gym,Gym / Fitness Center,Hawaiian Restaurant,Hostel,Hotel,Hotel Bar,Hotpot Restaurant,Ice Cream Shop,Indian Restaurant,Italian Restaurant,Japanese Restaurant,Jazz Club,Jewelry Store,Juice Bar,Karaoke Bar,Korean Restaurant,Lounge,Market,Massage Studio,Mattress Store,Mexican Restaurant,Middle Eastern Restaurant,Movie Theater,Multiplex,Museum,Music Venue,Noodle House,North Indian Restaurant,Opera House,Outdoors & Recreation,Park,Pier,Pizza Place,Pub,Ramen Restaurant,Residential Building (Apartment / Condo),Rest Area,Restaurant,Russian Restaurant,Sandwich Place,Seafood Restaurant,Shoe Store,Shopping Mall,Shopping Plaza,Snack Place,Soup Place,Spa,Spanish Restaurant,Speakeasy,Steakhouse,Supermarket,Sushi Restaurant,Tapas Restaurant,Tattoo Parlor,Tea Room,Temple,Thai Restaurant,Theme Park,Track Stadium,Travel Agency,Turkish Restaurant,University,Vegetarian / Vegan Restaurant,Video Game Store,Vietnamese Restaurant,Warehouse Store,Water Park,Whisky Bar,Women's Store,Yoga Studio
0,Bình Chánh District,0.00000,0.000000,0.00000,0.000000,0.166667,0.000000,0.000000,0.000000,0.166667,0.000000,0.000,0.000000,0.00,0.00,0.00,0.000000,0.00,0.000,0.000000,0.00000,0.000000,0.000000,0.00,0.166667,0.000000,0.000000,0.000000,0.00,0.00,0.000000,0.000000,0.00,0.00,0.000000,0.000000,0.000000,0.000000,0.166667,0.000000,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000,0.00,0.000000,0.00,0.000000,0.000000,0.00,0.00,0.00000,0.00000,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.00000,0.00,0.00,0.00,0.000000,0.00,0.00,0.00,0.000000,0.000000,0.00,0.000000,0.00,0.000,0.00,0.00,0.000000,0.000000,0.00,0.00,0.000000,0.00,0.00,0.00,0.000000,0.000000,0.000000,0.00,0.00,0.000000,0.000,0.000000,0.000000,0.000000,0.166667,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.00,0.000000,0.000000,0.000000,0.00,0.00,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.000,0.000000,0.000000,0.00,0.000000,0.00000,0.000000,0.00,0.166667,0.000000
1,Bình Thạnh District,0.00000,0.000000,0.00000,0.000000,0.000000,0.041667,0.000000,0.013889,0.000000,0.013889,0.000,0.000000,0.00,0.00,0.00,0.013889,0.00,0.000,0.013889,0.00000,0.027778,0.000000,0.00,0.000000,0.194444,0.000000,0.000000,0.00,0.00,0.111111,0.000000,0.00,0.00,0.000000,0.013889,0.013889,0.013889,0.000000,0.000000,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000,0.00,0.000000,0.00,0.013889,0.000000,0.00,0.00,0.00000,0.00000,0.000000,0.000000,0.013889,0.00,0.000000,0.000000,0.000000,0.00000,0.00,0.00,0.00,0.013889,0.00,0.00,0.00,0.000000,0.013889,0.00,0.000000,0.00,0.000,0.00,0.00,0.000000,0.013889,0.00,0.00,0.013889,0.00,0.00,0.00,0.013889,0.000000,0.000000,0.00,0.00,0.000000,0.000,0.013889,0.013889,0.000000,0.041667,0.00,0.013889,0.000000,0.027778,0.000000,0.013889,0.00,0.00,0.000000,0.013889,0.000000,0.00,0.00,0.000000,0.000000,0.00,0.000000,0.000000,0.013889,0.000,0.000000,0.055556,0.00,0.208333,0.00000,0.000000,0.00,0.000000,0.013889
2,"Bình Tân District, Ho Chi Minh City",0.00000,0.000000,0.00000,0.058824,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000,0.000000,0.00,0.00,0.00,0.000000,0.00,0.000,0.000000,0.00000,0.000000,0.000000,0.00,0.000000,0.294118,0.000000,0.000000,0.00,0.00,0.117647,0.000000,0.00,0.00,0.058824,0.000000,0.000

In [32]:
len(hcmc_grouped[hcmc_grouped["Shopping Mall"] > 0])

5

**Create a new DataFrame for Shopping Mall data only**

In [34]:
hcmc_mall = hcmc_grouped[["Neighborhoods","Shopping Mall"]]

In [35]:
hcmc_mall.head()

,Neighborhoods,Shopping Mall
0,Bình Chánh District,0.000000
1,Bình Thạnh District,0.013889
2,"Bình Tân District, Ho Chi Minh City",0.117647
3,Cần Giờ District,0.000000
4,Củ Chi District,0.000000


### 7. Cluster Neighborhoods
Run k-means to cluster the neighborhoods in Kuala Lumpur into 3 clusters.

In [38]:
# set number of clusters
k = 3

hcmc_clustering = hcmc_mall.drop(["Neighborhoods"], 1)

# run k-means clustering
kmeans = KMeans(n_clusters=k, random_state=0).fit(hcmc_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10] 

array([0, 0, 1, 0, 0, 0, 0, 0, 0, 0], dtype=int32)

In [39]:
# create a new dataframe that includes the cluster as well as the top 10 venues for each neighborhood.
hcmc_merged = hcmc_mall.copy()

# add clustering labels
hcmc_merged["Cluster Labels"] = kmeans.labels_

In [40]:
hcmc_merged.rename(columns={"Neighborhoods": "Neighborhood"}, inplace=True)
hcmc_merged.head()

,Neighborhood,Shopping Mall,Cluster Labels
0,Bình Chánh District,0.000000,0
1,Bình Thạnh District,0.013889,0
2,"Bình Tân District, Ho Chi Minh City",0.117647,1
3,Cần Giờ District,0.000000,0
4,Củ Chi District,0.000000,0


In [42]:
# merge toronto_grouped with toronto_data to add latitude/longitude for each neighborhood
hcmc_merged = hcmc_merged.join(df.set_index("Neighborhood"), on="Neighborhood")

print(hcmc_merged.shape)
hcmc_merged.head() # check the last columns!

(23, 5)


,Neighborhood,Shopping Mall,Cluster Labels,Latitude,Longitude
0,Bình Chánh District,0.000000,0,10.67922,106.57654
1,Bình Thạnh District,0.013889,0,10.80608,106.69297
2,"Bình Tân District, Ho Chi Minh City",0.117647,1,10.73684,106.61448
3,Cần Giờ District,0.000000,0,10.41566,106.96130
4,Củ Chi District,0.000000,0,10.97734,106.50223


In [44]:
# sort the results by Cluster Labels
print(hcmc_merged.shape)
hcmc_merged.sort_values(["Cluster Labels"], inplace=True)
hcmc_merged

(23, 5)


,Neighborhood,Shopping Mall,Cluster Labels,Latitude,Longitude
0,Bình Chánh District,0.000000,0,10.67922,106.576540
20,Thủ Đức,0.000000,0,10.86178,106.796107
19,Template:List of HCMC Administrative Units,0.000000,0,10.77641,106.701110
18,Phú Nhuận District,0.000000,0,10.79565,106.674640
17,Nhà Bè District,0.000000,0,10.70153,106.738180
16,Hóc Môn District,0.000000,0,10.88836,106.596400
14,"District 8, Ho Chi Minh City",0.000000,0,10.74771,106.663340
13,"District 7, Ho Chi Minh City",0.000000,0,10.70515,106.737480
12,"District 6, Ho Chi Minh City",0.000000,0,10.74597,106.647690
10,"District 4, Ho Chi Minh City",0.000000,0,10.76670,106.706470


**Finally, let's visualize the resulting clusters**

In [45]:
# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(k)
ys = [i+x+(i*x)**2 for i in range(k)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(hcmc_merged['Latitude'], hcmc_merged['Longitude'], hcmc_merged['Neighborhood'], hcmc_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' - Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

In [ ]:
# save the map as HTML file
map_clusters.save('map_clusters.html')

### 8. Examine Clusters

#### Cluster 0

In [46]:
hcmc_merged.loc[hcmc_merged['Cluster Labels'] == 0]

,Neighborhood,Shopping Mall,Cluster Labels,Latitude,Longitude
0,Bình Chánh District,0.000000,0,10.67922,106.576540
20,Thủ Đức,0.000000,0,10.86178,106.796107
19,Template:List of HCMC Administrative Units,0.000000,0,10.77641,106.701110
18,Phú Nhuận District,0.000000,0,10.79565,106.674640
17,Nhà Bè District,0.000000,0,10.70153,106.738180
16,Hóc Môn District,0.000000,0,10.88836,106.596400
14,"District 8, Ho Chi Minh City",0.000000,0,10.74771,106.663340
13,"District 7, Ho Chi Minh City",0.000000,0,10.70515,106.737480
12,"District 6, Ho Chi Minh City",0.000000,0,10.74597,106.647690
10,"District 4, Ho Chi Minh City",0.000000,0,10.76670,106.706470


#### Cluster 1

In [47]:
hcmc_merged.loc[hcmc_merged['Cluster Labels'] == 1]

,Neighborhood,Shopping Mall,Cluster Labels,Latitude,Longitude
21,Tân Bình District,0.117647,1,10.73684,106.61448
2,"Bình Tân District, Ho Chi Minh City",0.117647,1,10.73684,106.61448


#### Cluster 2

In [48]:
hcmc_merged.loc[hcmc_merged['Cluster Labels'] == 2]

,Neighborhood,Shopping Mall,Cluster Labels,Latitude,Longitude
15,Gò Vấp District,0.047619,2,10.83379,106.66556


#### Observations:
Most of the shopping malls are concentrated in the central area of HCM city, with the highest number in cluster 0. On the other hand, cluster 1 and 2 has very low number to totally no shopping mall in the neighborhoods. This represents a great opportunity and high potential areas to open new shopping malls as there is very little to no competition from existing malls. Meanwhile, shopping malls in cluster 0 are likely suffering from intense competition due to oversupply and high concentration of shopping malls. From another perspective, this also shows that the oversupply of shopping malls mostly happened in the central area of the city, with the suburb area still have very few shopping malls. Therefore, this project recommends property developers to capitalize on these findings to open new shopping malls in neighborhoods in cluster 2 with little to no competition. Property developers with unique selling propositions to stand out from the competition can also open new shopping malls in neighborhoods in cluster 1 with moderate competition. Lastly, property developers are advised to avoid neighborhoods in cluster 0 which already have high concentration of shopping malls and suffering from intense competition.